In [ ]:
import requests
URL = "https://open.neis.go.kr/portal/data/file/downloadBlobFileData.do?infId=OPEN15920190423094641415608&infSeq=3&fileSeq=35"
  
file = requests.get(URL, stream = True)
  
with open("학원교습소정보.csv","wb") as pdf:
    for chunk in file.iter_content(chunk_size=1024):
  
         if chunk:
             pdf.write(chunk)

In [ ]:
import json

school_data_to_load = "전국초중등학교위치표준데이터.json"

with open(school_data_to_load, encoding='UTF8') as json_file:
    json_data = json.load(json_file)

In [ ]:
#고등학교만 골라내기
schools = []
for data in json_data['records']:
    if data['학교명'].find('고등학교') != -1:
        schools.append(data)

In [ ]:
#사립/공립 분리
private_schools = {}
public_schools = {}

for school in schools:
    if school['설립형태'].find('사립') != -1:
        if school['시도교육청명'] in private_schools:
            private_schools[school['시도교육청명']].append(school)
        else:
            private_schools[school['시도교육청명']] = [school]
    else:
        if school['시도교육청명'] in public_schools:
            public_schools[school['시도교육청명']].append(school)
        else:
            public_schools[school['시도교육청명']] = [school]

In [ ]:
# 학원 정보
import pandas as pd
academy_data_to_load = "학원교습소정보_2023년04월30일기준.csv"

_academy_data = pd.read_csv(academy_data_to_load, encoding='cp949')

In [ ]:
# 학원 지역별 분류

academy_data = {}
for index, row in _academy_data.iterrows():
    if row['시도교육청명'] in academy_data:
        academy_data[row['시도교육청명']].append(row)
    else:
        academy_data[row['시도교육청명']] = [row]

In [ ]:
# 학원 주소로 좌표 구하기
import requests
import json

url_front = "http://api.vworld.kr/req/address?"
url_params = "service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type=road"
url_address = "&address="
url_key = "&key="
auth_key = "86669EDB-62D6-34DA-A1E4-6B2E8948946B"

location_data = {}


for location in academy_data:
    cnt = 1
    for academy in academy_data[location]:
        address = academy['도로명주소']
        try:
            url = url_front + url_params + url_address + address + url_key + auth_key
            result = requests.get(url)
            json_data = result.json()

            if json_data['response']['status'] == 'OK':
                x = json_data['response']['result']['point']['x']
                y = json_data['response']['result']['point']['y']
                if location in location_data:
                    location_data[location].append([x, y])
                else:
                    location_data[location] = [[x, y]]
                if cnt % 500 == 0:
                    print(cnt)
                cnt += 1
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(e)
            pass


In [ ]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    두 좌표 간의 Haversine 거리를 계산하는 함수
    """
    # 지구의 반지름 (단위: km)
    earth_radius = 6371

    # 각도를 라디안으로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # 좌표 간의 차이 계산
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine 공식을 이용한 거리 계산
    a = math.sin(delta_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = earth_radius * c

    return distance

def find_nearby_academies(school_lat, school_lon, academies, radius):
    """
    주어진 학교 위치 주변에 있는 학원들을 찾는 함수
    """
    nearby_academies = []

    for academy in academies:
        academy_lon = float(academy[0])
        academy_lat = float(academy[1])

        distance = haversine_distance(school_lat, school_lon, academy_lat, academy_lon)
        if distance <= radius:
            nearby_academies.append(academy)

    return nearby_academies


In [ ]:
# 각 시도 교육청별로 공립/사립 학교의 1km 반경 내에 있는 학원 수의 평균을 비교합니다.
for key in location_data:
    print(key)
    #사립 평균
    sum = 0
    for school in private_schools:
        school_lat = float(school['위도'])
        school_lon = float(school['경도'])
        academies = location_data[key]
        nearby_academies = find_nearby_academies(school_lat, school_lon, academies, 1)
        sum += len(nearby_academies)
    print(f"{key} 사립 평균: {sum / len(private_schools)}")
    #공립 평균
    sum = 0
    for school in public_schools:
        school_lat = float(school['위도'])
        school_lon = float(school['경도'])
        academies = location_data[key]
        nearby_academies = find_nearby_academies(school_lat, school_lon, academies, 1)
        sum += len(nearby_academies)
    print(f"{key} 공립 평균: {sum / len(public_schools)}")